# T1. dataset 和 vocabulary 的基本使用

&emsp; 1 &ensp; dataset 的使用与结构
 
&emsp; &emsp; 1.1 &ensp; dataset 的结构与创建

&emsp; &emsp; 1.2 &ensp; dataset 的数据预处理

&emsp; &emsp; 1.3 &ensp; 延伸：instance 和 field

&emsp; 2 &ensp; vocabulary 的结构与使用

&emsp; &emsp; 2.1 &ensp; vocabulary 的创建与修改

&emsp; &emsp; 2.2 &ensp; vocabulary 与 OOV 问题

&emsp; 3 &ensp; dataset 和 vocabulary 的组合使用
 
&emsp; &emsp; 3.1 &ensp; 从 dataframe 中加载 dataset

&emsp; &emsp; 3.2 &ensp; 从 dataset 中获取 vocabulary

## 1. dataset 的基本使用

### 1.1  dataset 的结构与创建

在`fastNLP 0.8`中，使用`DataSet`模块表示数据集，**`dataset`类似于关系型数据库中的数据表**（下文统一为小写`dataset`）

&emsp; **主要包含`field`字段和`instance`实例两个元素**，对应`table`中的`field`字段和`record`记录

在`fastNLP 0.8`中，`DataSet`模块被定义在`fastNLP.core.dataset`路径下，导入该模块后，最简单的

&emsp; 初始化方法，即将字典形式的表格 **`{'field1': column1, 'field2': column2, ...}`** 传入构造函数

In [1]:
from fastNLP.core.dataset import DataSet

data = {'idx': [0, 1, 2],  
        'sentence':["This is an apple .", "I like apples .", "Apples are good for our health ."],
        'words': [['This', 'is', 'an', 'apple', '.'], 
                  ['I', 'like', 'apples', '.'], 
                  ['Apples', 'are', 'good', 'for', 'our', 'health', '.']],
        'num': [5, 4, 7]}

dataset = DataSet(data)
print(dataset)

+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .        | ['I', 'like', 'appl... | 4   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


&emsp; 在`dataset`的实例中，字段`field`的名称和实例`instance`中的字符串也可以中文

In [2]:
temp = {'序号': [0, 1, 2],  
        '句子':["生活就像海洋，", "只有意志坚强的人，", "才能到达彼岸。"],
        '字符': [['生', '活', '就', '像', '海', '洋', '，'], 
                  ['只', '有', '意', '志', '坚', '强', '的', '人', '，'], 
                  ['才', '能', '到', '达', '彼', '岸', '。']],
        '长度': [7, 9, 7]}

chinese = DataSet(temp)
print(chinese)

+------+--------------------+------------------------+------+
| 序号 | 句子               | 字符                   | 长度 |
+------+--------------------+------------------------+------+
| 0    | 生活就像海洋，     | ['生', '活', '就', ... | 7    |
| 1    | 只有意志坚强的人， | ['只', '有', '意', ... | 9    |
| 2    | 才能到达彼岸。     | ['才', '能', '到', ... | 7    |
+------+--------------------+------------------------+------+


在`dataset`中，使用`drop`方法可以删除满足条件的实例，这里使用了python中的`lambda`表达式

&emsp; 注一：在`drop`方法中，通过设置`inplace`参数将删除对应实例后的`dataset`作为一个新的实例生成

In [3]:
dropped = dataset
dropped = dropped.drop(lambda ins:ins['num'] < 5, inplace=False)
print(id(dropped), id(dataset))
print(dropped)
print(dataset)

1969418794120 1971237588872
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .        | ['I', 'like', 'appl... | 4   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


&emsp; 注二：在`fastNLP 0.8`中，**对`dataset`使用等号**，**其效果是传引用**，**而不是赋值**（？？？）

&emsp; &emsp; 如下所示，**`dropped`和`dataset`具有相同`id`**，**对`dropped`执行删除操作`dataset`同时会被修改**

In [4]:
dropped = dataset
dropped.drop(lambda ins:ins['num'] < 5)
print(id(dropped), id(dataset))
print(dropped)
print(dataset)

1971237588872 1971237588872
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


在`dataset`中，使用`delet_instance`方法可以删除对应序号的`instance`实例，序号从0开始

In [5]:
dataset = DataSet(data)
dataset.delete_instance(2)
print(dataset)

+-----+--------------------+------------------------+-----+
| idx | sentence           | words                  | num |
+-----+--------------------+------------------------+-----+
| 0   | This is an apple . | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .    | ['I', 'like', 'appl... | 4   |
+-----+--------------------+------------------------+-----+


在`dataset`中，使用`delet_field`方法可以删除对应名称的`field`字段

In [6]:
dataset.delete_field('num')
print(dataset)

+-----+--------------------+------------------------------+
| idx | sentence           | words                        |
+-----+--------------------+------------------------------+
| 0   | This is an apple . | ['This', 'is', 'an', 'app... |
| 1   | I like apples .    | ['I', 'like', 'apples', '... |
+-----+--------------------+------------------------------+


### 1.2  dataset 的数据预处理

在`dataset`模块中，`apply`、`apply_field`、`apply_more`和`apply_field_more`函数可以进行简单的数据预处理

&emsp; **`apply`和`apply_more`针对整条实例**，**`apply_field`和`apply_field_more`仅针对实例的部分字段**

&emsp; **`apply`和`apply_field`仅针对单个字段**，**`apply_more`和`apply_field_more`则可以针对多个字段**

&emsp; **`apply`和`apply_field`返回的是个列表**，**`apply_more`和`apply_field_more`返回的是个字典**

***

`apply`的参数包括一个函数`func`和一个新字段名`new_field_name`，函数`func`的处理对象是`dataset`模块中

&emsp; 的每个`instance`实例，函数`func`的处理结果存放在`new_field_name`对应的新建字段内

In [7]:
data = {'idx': [0, 1, 2], 
        'sentence':["This is an apple .", "I like apples .", "Apples are good for our health ."], }
dataset = DataSet(data)
dataset.apply(lambda ins: ins['sentence'].split(), new_field_name='words')
print(dataset)

Output()

+-----+------------------------------+------------------------------+
| idx | sentence                     | words                        |
+-----+------------------------------+------------------------------+
| 0   | This is an apple .           | ['This', 'is', 'an', 'app... |
| 1   | I like apples .              | ['I', 'like', 'apples', '... |
| 2   | Apples are good for our h... | ['Apples', 'are', 'good',... |
+-----+------------------------------+------------------------------+


&emsp; **`apply`使用的函数可以是一个基于`lambda`表达式的匿名函数**，**也可以是一个自定义的函数**

In [8]:
dataset = DataSet(data)

def get_words(instance):
    sentence = instance['sentence']
    words = sentence.split()
    return words

dataset.apply(get_words, new_field_name='words')
print(dataset)

+-----+------------------------------+------------------------------+
| idx | sentence                     | words                        |
+-----+------------------------------+------------------------------+
| 0   | This is an apple .           | ['This', 'is', 'an', 'app... |
| 1   | I like apples .              | ['I', 'like', 'apples', '... |
| 2   | Apples are good for our h... | ['Apples', 'are', 'good',... |
+-----+------------------------------+------------------------------+


`apply_field`的参数，除了函数`func`外还有`field_name`和`new_field_name`，该函数`func`的处理对象仅

&emsp; 是`dataset`模块中的每个`field_name`对应的字段内容，处理结果存放在`new_field_name`对应的新建字段内

In [9]:
dataset = DataSet(data)
dataset.apply_field(lambda sent:sent.split(), field_name='sentence', new_field_name='words')
print(dataset)

+-----+------------------------------+------------------------------+
| idx | sentence                     | words                        |
+-----+------------------------------+------------------------------+
| 0   | This is an apple .           | ['This', 'is', 'an', 'app... |
| 1   | I like apples .              | ['I', 'like', 'apples', '... |
| 2   | Apples are good for our h... | ['Apples', 'are', 'good',... |
+-----+------------------------------+------------------------------+


`apply_more`的参数只有函数`func`，函数`func`的处理对象是`dataset`模块中的每个`instance`实例

&emsp; 要求函数`func`返回一个字典，根据字典的`key-value`确定存储在`dataset`中的字段名称与内容

In [10]:
dataset = DataSet(data)
dataset.apply_more(lambda ins:{'words': ins['sentence'].split(), 'num': len(ins['sentence'].split())})
print(dataset)

+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .        | ['I', 'like', 'appl... | 4   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


`apply_more`的参数只有函数`func`，函数`func`的处理对象是`dataset`模块中的每个`instance`实例

&emsp; 要求函数`func`返回一个字典，根据字典的`key-value`确定存储在`dataset`中的字段名称与内容

In [11]:
dataset = DataSet(data)
dataset.apply_field_more(lambda sent:{'words': sent.split(), 'num': len(sent.split())}, 
                         field_name='sentence')
print(dataset)

+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .        | ['I', 'like', 'appl... | 4   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


### 1.3 延伸：instance 和 field

在`fastNLP 0.8`中，使用`Instance`模块表示数据集`dataset`中的每条数据，被称为实例

&emsp; 构造方式类似于构造一个字典，通过键值相同的`Instance`列表，也可以初始化一个`dataset`，代码如下

In [12]:
from fastNLP.core.dataset import DataSet
from fastNLP.core.dataset import Instance

dataset = DataSet([
    Instance(sentence="This is an apple .",
        words=['This', 'is', 'an', 'apple', '.'],
        num=5),
    Instance(sentence="I like apples .",
        words=['I', 'like', 'apples', '.'],
        num=4),
    Instance(sentence="Apples are good for our health .",
        words=['Apples', 'are', 'good', 'for', 'our', 'health', '.'],
        num=7),
    ])

&emsp; 通过`items`、`keys`和`values`方法，可以分别获得`dataset`的`item`列表、`key`列表、`value`列表

In [13]:
ins = Instance(sentence="This is an apple .", words=['This', 'is', 'an', 'apple', '.'], num=5)

print(ins.items())
print(ins.keys())
print(ins.values())

dict_items([('sentence', 'This is an apple .'), ('words', ['This', 'is', 'an', 'apple', '.']), ('num', 5)])
dict_keys(['sentence', 'words', 'num'])
dict_values(['This is an apple .', ['This', 'is', 'an', 'apple', '.'], 5])


&emsp; 通过`add_field`方法，可以在`Instance`实例中，通过参数`field_name`添加字段，通过参数`field`赋值

In [14]:
ins.add_field(field_name='idx', field=0)
print(ins)

+--------------------+------------------------+-----+-----+
| sentence           | words                  | num | idx |
+--------------------+------------------------+-----+-----+
| This is an apple . | ['This', 'is', 'an'... | 5   | 0   |
+--------------------+------------------------+-----+-----+


在`fastNLP 0.8`中，使用`FieldArray`模块表示数据集`dataset`中的每条字段名（注：没有`field`类）

&emsp; 通过`get_all_fields`方法可以获取`dataset`的字段列表

&emsp; 通过`get_field_names`方法可以获取`dataset`的字段名称列表，代码如下

In [15]:
dataset.get_all_fields()

{'sentence': <fastNLP.core.dataset.field.FieldArray at 0x1ca8a879d08>,
 'words': <fastNLP.core.dataset.field.FieldArray at 0x1ca8a879d88>,
 'num': <fastNLP.core.dataset.field.FieldArray at 0x1ca8a879e08>}

In [16]:
dataset.get_field_names()

['num', 'sentence', 'words']

其他`dataset`的基本使用：通过`in`或者`has_field`方法可以判断`dataset`的是否包含某种字段

&emsp; 通过`rename_field`方法可以更改`dataset`中的字段名称；通过`concat`方法可以实现两个`dataset`中的拼接

&emsp; 通过`len`可以统计`dataset`中的实例数目；`dataset`的全部变量与函数可以通过`dir(dataset)`查询

In [17]:
print(len(dataset), dataset.has_field('length')) 
if 'num' in dataset:
    dataset.rename_field('num', 'length')
elif 'length' in dataset:
    dataset.rename_field('length', 'num')
dataset.concat(dataset)
print(len(dataset), dataset.has_field('length'))  
print(dataset)  

3 False
6 True
+------------------------------+------------------------------+--------+
| sentence                     | words                        | length |
+------------------------------+------------------------------+--------+
| This is an apple .           | ['This', 'is', 'an', 'app... | 5      |
| I like apples .              | ['I', 'like', 'apples', '... | 4      |
| Apples are good for our h... | ['Apples', 'are', 'good',... | 7      |
| This is an apple .           | ['This', 'is', 'an', 'app... | 5      |
| I like apples .              | ['I', 'like', 'apples', '... | 4      |
| Apples are good for our h... | ['Apples', 'are', 'good',... | 7      |
+------------------------------+------------------------------+--------+


## 2. vocabulary 的结构与使用

### 2.1 vocabulary 的创建与修改

在`fastNLP 0.8`中，使用`Vocabulary`模块表示词汇表，**`vocabulary`的核心是从单词到序号的映射**

&emsp; 可以直接通过构造函数实例化，通过查找`word2idx`属性，可以找到`vocabulary`映射对应的字典实现

&emsp; **默认补零`padding`用`<pad>`表示**，**对应序号为0**；**未知单词`unknown`用`<unk>`表示**，**对应序号1**

&emsp; 通过打印`vocabulary`可以看到词汇表中的单词列表，其中，`padding`和`unknown`不会显示

In [18]:
from fastNLP.core.vocabulary import Vocabulary

vocab = Vocabulary()
print(vocab)
print(vocab.word2idx)
print(vocab.padding, vocab.padding_idx)
print(vocab.unknown, vocab.unknown_idx)

Vocabulary([]...)
{'<pad>': 0, '<unk>': 1}
<pad> 0
<unk> 1


在`vocabulary`中，通过`add_word`方法或`add_word_lst`方法，可以单独或批量添加单词

&emsp; 通过`len`或`word_count`属性，可以显示`vocabulary`的单词量和每个单词添加的次数

In [19]:
vocab.add_word_lst(['生活', '就像', '海洋'])
print(len(vocab), vocab.word_count)
vocab.add_word('只有')
print(len(vocab), vocab.word_count)

5 Counter({'生活': 1, '就像': 1, '海洋': 1})
6 Counter({'生活': 1, '就像': 1, '海洋': 1, '只有': 1})


&emsp; **通过`to_word`方法可以找到单词对应的序号**，**通过`to_index`方法可以找到序号对应的单词**

&emsp; &emsp; 由于序号0和序号1已经被占用，所以**新加入的词的序号从2开始计数**，如`'生活'`对应2

&emsp; &emsp; 通过`has_word`方法可以判断单词是否在词汇表中，没有的单词被判做`<unk>`

In [20]:
print(vocab.to_word(0), vocab.to_index('<pad>'))
print(vocab.to_word(1), vocab.to_index('<unk>'))
print(vocab.to_word(2), vocab.to_index('生活'))
print(vocab.to_word(5), vocab.to_index('只有'))
print('彼岸', vocab.to_index('彼岸'), vocab.has_word('彼岸'))

<pad> 0
<unk> 1
生活 2
只有 5
彼岸 1 False


**`vocabulary`允许反复添加相同单词**，**可以通过`word_count`方法看到相应单词被添加的次数**

&emsp; 但其中没有`<unk>`和`<pad>`，`vocabulary`的全部变量与函数可以通过`dir(vocabulary)`查询

In [21]:
vocab.add_word_lst(['生活', '就像', '海洋', '只有', '意志', '坚强的', '人', '才', '能', '到达', '彼岸'])
print(len(vocab), vocab.word_count)
print('彼岸', vocab.to_index('彼岸'), vocab.has_word('彼岸'))

13 Counter({'生活': 2, '就像': 2, '海洋': 2, '只有': 2, '意志': 1, '坚强的': 1, '人': 1, '才': 1, '能': 1, '到达': 1, '彼岸': 1})
彼岸 12 True


### 2.2 vocabulary 与 OOV 问题

在`vocabulary`模块初始化的时候，可以通过指定`unknown`和`padding`为`None`，限制其存在

&emsp; 此时添加单词直接从0开始标号，如果遇到未知单词会直接报错，即 out of vocabulary

In [22]:
vocab = Vocabulary(unknown=None, padding=None)

vocab.add_word_lst(['positive', 'negative'])
print(vocab.word2idx)

try:
    print(vocab.to_index('neutral'))
except ValueError:
    print("ValueError: word `neutral` not in vocabulary")

{'positive': 0, 'negative': 1}
ValueError: word `neutral` not in vocabulary


&emsp; 相应的，如果只指定其中的`unknown`，则编号会后移一个，同时遇到未知单词全部当做`<unk>`

In [23]:
vocab = Vocabulary(unknown='<unk>', padding=None)

vocab.add_word_lst(['positive', 'negative'])
print(vocab.word2idx)

print(vocab.to_index('neutral'), vocab.to_word(vocab.to_index('neutral')))

{'<unk>': 0, 'positive': 1, 'negative': 2}
0 <unk>


## 3 dataset 和 vocabulary 的组合使用
 
### 3.1 从 dataframe 中加载 dataset



### 3.2 从 dataset 中获取 vocabulary